In [1]:
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import gc
import xgboost as xgb

In [2]:
import gensim
from gensim.models import word2vec, KeyedVectors

In [3]:
inputPath = '../input/newSplited/'
outputPath = '../output/newSplited/'

In [4]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}

In [5]:
TOPN_candidate = 50
SET = 3
predictType = 'clicks'

In [8]:
note = 'cand100'
sub = f'_{0}'
noteModel = 'addRankofRule'
noteSubmission = 'cand100'
savedModelName = "cand100"
modelName = 'xgb_' + predictType + '_' + noteModel
modelSavedPAth = outputPath + f'savedModel_set2/{modelName}/'

# Online predict

In [10]:
data4xgb_path = outputPath + f'data4xgb_set{SET}_{note}/'
data4xgb = pd.read_parquet(data4xgb_path + f'data4xgb_{predictType}{sub}.pqt')
data4xgb

,session,aid,order_by_rule,item_item_count,item_user_count,item_buy_ratio,item_type_std,item_ts_mean,item_ts_std,item_clicked_cnt,...,user_buy_ratio,user_type_std,user_ts_mean,user_ts_std,user_clicked_cnt,user_carted_cnt,user_ordered_cnt,cnt_clicked,cnt_carted,cnt_ordered
0,12899779,59625,99,0.000073,0.000151,0.000000,0.000000,1141376.0,875594.1875,0.000077,...,0.0,0.0,0.0,0.000000,0.002309,0.0,0.0,1.0,0.0,0.0
1,12899779,1253524,98,0.002002,0.002899,0.021505,0.145322,1354752.0,706245.1875,0.002083,...,0.0,0.0,0.0,0.000000,0.002309,0.0,0.0,0.0,0.0,0.0
2,12899779,737445,97,0.006934,0.008229,0.016684,0.128152,1384960.0,765651.3125,0.007253,...,0.0,0.0,0.0,0.000000,0.002309,0.0,0.0,0.0,0.0,0.0
3,12899779,438191,96,0.026460,0.031651,0.033416,0.185741,1140480.0,763489.2500,0.027237,...,0.0,0.0,0.0,0.000000,0.002309,0.0,0.0,0.0,0.0,0.0
4,12899779,731692,95,0.000943,0.000824,0.203008,0.488650,1252608.0,834912.2500,0.000833,...,0.0,0.0,0.0,0.000000,0.002309,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49999995,13399778,1685214,4,0.334545,0.266382,0.144687,0.425379,1396864.0,707636.6875,0.314571,...,0.0,0.0,161152.0,11.313708,0.004619,0.0,0.0,0.0,0.0,0.0
49999996,13399778,1673641,3,0.136098,0.160164,0.054724,0.259407,1438336.0,788280.2500,0.137984,...,0.0,0.0,161152.0,11.313708,0.004619,0.0,0.0,0.0,0.0,0.0
49999997,13399778,909449,2,0.247101,0.260076,0.053359,0.251499,1376640.0,751218.7500,0.250513,...,0.0,0.0,161152.0,11.313708,0.004619,0.0,0.0,0.0,0.0,0.0
49999998,13399778,1260564,1,0.022129,0.036679,0.049443,0.259465,2000384.0,698647.3750,0.022615,...,0.0,0.0,161152.0,11.313708,0.004619,0.0,0.0,0.0,0.0,0.0


## Add w2v features

In [11]:
aid2vec_model = gensim.models.KeyedVectors.load_word2vec_format('../output/newSplited/savedModel/otto_aid2vec_5d.bin', binary=True)

In [12]:
aid2vec = {}
for aid in aid2vec_model.index_to_key:
    aid2vec[int(aid)] = aid2vec_model[aid]

In [13]:
aid2vec_df = pd.DataFrame.from_dict(aid2vec, orient='index').reset_index()
aid2vec_df.columns=['aid', 'd0', 'd1', 'd2', 'd3', 'd4']
aid2vec_df

,aid,d0,d1,d2,d3,d4
0,1460571,-0.164300,0.109582,0.061829,-0.024449,-0.026794
1,485256,0.143439,-0.165622,0.078947,-0.119412,-0.162329
2,108125,0.010592,0.190249,0.094301,0.104456,0.086991
3,29735,0.114426,0.005291,-0.148755,0.135899,-0.019846
4,1733943,0.000141,-0.051681,-0.126980,0.170706,0.112627
...,...,...,...,...,...,...
1855598,753559,-0.038612,0.014407,-0.027756,-0.189586,-0.015147
1855599,1317675,-0.079194,-0.154401,0.037340,0.122228,-0.171527
1855600,1166769,0.197370,-0.162120,-0.189011,-0.112653,-0.044016
1855601,276785,0.020701,-0.031155,0.181024,-0.033857,-0.118816


In [14]:
data4xgb = data4xgb.merge(aid2vec_df, on=['aid'], how='left')
data4xgb

,session,aid,order_by_rule,item_item_count,item_user_count,item_buy_ratio,item_type_std,item_ts_mean,item_ts_std,item_clicked_cnt,...,user_carted_cnt,user_ordered_cnt,cnt_clicked,cnt_carted,cnt_ordered,d0,d1,d2,d3,d4
0,12899779,59625,99,0.000073,0.000151,0.000000,0.000000,1141376.0,875594.1875,0.000077,...,0.0,0.0,1.0,0.0,0.0,0.148760,-0.048975,0.191587,-0.154023,0.148203
1,12899779,1253524,98,0.002002,0.002899,0.021505,0.145322,1354752.0,706245.1875,0.002083,...,0.0,0.0,0.0,0.0,0.0,-0.039628,-0.050114,-0.038292,-0.051404,-0.182693
2,12899779,737445,97,0.006934,0.008229,0.016684,0.128152,1384960.0,765651.3125,0.007253,...,0.0,0.0,0.0,0.0,0.0,-0.112163,-0.139412,0.012132,0.100322,-0.013511
3,12899779,438191,96,0.026460,0.031651,0.033416,0.185741,1140480.0,763489.2500,0.027237,...,0.0,0.0,0.0,0.0,0.0,-0.147640,-0.040210,0.129615,0.039616,0.146490
4,12899779,731692,95,0.000943,0.000824,0.203008,0.488650,1252608.0,834912.2500,0.000833,...,0.0,0.0,0.0,0.0,0.0,0.123628,-0.145712,0.082015,0.066697,0.072692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49999995,13399778,1685214,4,0.334545,0.266382,0.144687,0.425379,1396864.0,707636.6875,0.314571,...,0.0,0.0,0.0,0.0,0.0,0.123039,-0.093210,0.181509,-0.067372,0.018002
49999996,13399778,1673641,3,0.136098,0.160164,0.054724,0.259407,1438336.0,788280.2500,0.137984,...,0.0,0.0,0.0,0.0,0.0,0.133012,0.030254,-0.129457,-0.137710,-0.030602
49999997,13399778,909449,2,0.247101,0.260076,0.053359,0.251499,1376640.0,751218.7500,0.250513,...,0.0,0.0,0.0,0.0,0.0,-0.026553,-0.008849,-0.133952,-0.033244,0.041671
49999998,13399778,1260564,1,0.022129,0.036679,0.049443,0.259465,2000384.0,698647.3750,0.022615,...,0.0,0.0,0.0,0.0,0.0,0.096555,-0.111347,0.154345,-0.053869,0.012431


## Normalize features

In [12]:
# data4xgb.iloc[:, 2:] = data4xgb.iloc[:, 2:] - data4xgb.iloc[:, 2:].min()
# data4xgb.iloc[:, 2:] = data4xgb.iloc[:, 2:] / data4xgb.iloc[:, 2:].max()
# data4xgb

In [16]:
modelSavedPAth + modelName + f'_fold{fold}.xgb'

'../output/newSplited/savedModel_set2/xgb_clicks__addRankofRule/xgb_clicks__addRankofRule_fold0.xgb'

In [18]:
preds = np.zeros(len(data4xgb))
for fold in range(5):
    print('======= fold', fold, '===========')
    model = xgb.Booster()
    model.load_model(modelSavedPAth + modelName + f'_fold{fold}.xgb')
    model.set_param({'predictor': 'gpu_predictor'})

    chunkSize = 10000000
    n = len(data4xgb)
    i = 0
    pred = np.array([])
    while (i * chunkSize < n):
        dtest = xgb.DMatrix(data4xgb.iloc[i*chunkSize:(i+1)*chunkSize, 2:])
        pred = np.append(pred, model.predict(dtest))
        i += 1
        
    preds += pred/5
predictions = data4xgb[['session','aid']].copy()
predictions['pred'] = preds
predictions

======= fold 0 ===========
======= fold 1 ===========
======= fold 2 ===========
======= fold 3 ===========
======= fold 4 ===========


,session,aid,pred
0,12899779,59625,3.772850
1,12899779,1253524,3.362456
2,12899779,737445,3.201696
3,12899779,438191,2.953659
4,12899779,731692,3.541986
...,...,...,...
49999995,13399778,1685214,-3.819172
49999996,13399778,1673641,-2.105783
49999997,13399778,909449,-2.459168
49999998,13399778,1260564,-1.835657


In [19]:
predictions = predictions.sort_values(['session','pred'], ascending=[True,False]).reset_index(drop=True)
predictions['n'] = predictions.groupby('session').aid.cumcount().astype('int8')
predictions = predictions.loc[predictions.n<20]
predictions

,session,aid,pred,n
0,12899779,59625,3.772850,0
1,12899779,637538,3.764667,1
2,12899779,985764,3.708089,2
3,12899779,1790770,3.643594,3
4,12899779,731692,3.541986,4
...,...,...,...,...
49999915,13399778,194980,2.421408,15
49999916,13399778,548262,2.356524,16
49999917,13399778,10565,2.352779,17
49999918,13399778,1095756,2.331899,18


In [15]:
sub = predictions.groupby('session').aid.apply(list).to_frame().reset_index()
sub.aid = sub.aid.apply(lambda x: " ".join(map(str,x)))
sub.columns = ['session_type','labels']
sub.session_type = sub.session_type.astype('str')+ '_' + predictType
sub

,session_type,labels
0,12899779_carts,59625 1790770 469285 159257 731692 1495817 397...
1,12899780_carts,736515 973453 582732 1142000 1495817 1493044 5...
2,12899781_carts,199008 141736 57315 918667 194067 461608 89326...
3,12899782_carts,779477 476063 975116 834354 229748 1696036 987...
4,12899783_carts,255297 1729553 300127 198385 1754419 1817895 1...
...,...,...
1671798,14571577_carts,1141710 1495817 970741 768043 86916 367734 934...
1671799,14571578_carts,519105 1495817 815460 822641 1580943 145332 14...
1671800,14571579_carts,739876 1495817 1550479 177070 702275 832213 85...
1671801,14571580_carts,202353 1495817 891417 1314576 679257 682237 98...


In [9]:
! mkdir {outputPath}predictions

In [16]:
sub.to_parquet(outputPath + 'predictions/' + modelName + '.pqt')

### integrate predictions

In [7]:
predictTypes = ['clicks', 'carts', 'orders']
subs = []
for i, predictType in enumerate(predictTypes):
    modelName = 'xgb_' + predictType + '_' + noteModel
    sub = pd.read_parquet(outputPath + 'predictions/' + modelName + '.pqt')
    subs.append(sub)
pred_df = pd.concat(subs).reset_index(drop=True)
pred_df
    

,session_type,labels
0,12899779_clicks,59625 637538 985764 1790770 731692 1660529 452...
1,12899780_clicks,1142000 1344758 973453 797055 1493044 1269469 ...
2,12899781_clicks,199008 1200570 89022 428697 461608 1583317 937...
3,12899782_clicks,889671 779477 975116 476063 413962 1711180 149...
4,12899783_clicks,1811433 58861 419088 562398 735071 255297 3001...
...,...,...
5015404,14571577_orders,1495817 970741 86916 145332 768043 631085 4803...
5015405,14571578_orders,1495817 145332 480314 1336175 471073 1609228 1...
5015406,14571579_orders,739876 1495817 857928 177070 832213 707204 179...
5015407,14571580_orders,202353 682237 679257 1495817 1314576 336919 14...


In [9]:
pred_df.to_csv(f'../submissions/Xgb_{noteSubmission}.csv', index=False)